In [17]:
from pyspark.mllib.util import MLUtils
from pyspark.sql import Row
import math
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from operator import mul
from operator import add
from pyspark.sql.functions import lit
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import when
from pyspark.ml.feature import VectorAssembler

In [4]:
df = spark.read.parquet("/Users/garlarohit/Desktop/tw623-nyu_ml/week4/mortgage_sample_week4/")
train = spark.read.parquet("/Users/garlarohit/Desktop/tw623-nyu_ml/week2/mortgage_sample")


In [19]:
trainP = train.withColumn("AGE1", when(train['age']> 20, train['age']).otherwise(20))
trainP = trainP.withColumn("AGE2", when(train['age']< 20, train['age']).otherwise(20))
train.show()

+-----+--------+--------------------+----+--------------+--------+----------+------+-----+--------------------+
| FICO|CLEANLTV|           INCENTIVE| AGE|FIRSTTIMEBUYER|INVESTOR|SECONDHOME|SEASON|label|            features|
+-----+--------+--------------------+----+--------------+--------+----------+------+-----+--------------------+
|751.0|    61.0|0.006200000000000004|19.0|           0.0|     0.0|       0.0|  12.0|    1|[751.0,61.0,0.006...|
|751.0|    61.0|0.008200000000000006|20.0|           0.0|     0.0|       0.0|   1.0|    1|[751.0,61.0,0.008...|
|751.0|    61.0|              0.0085|21.0|           0.0|     0.0|       0.0|   2.0|    2|[751.0,61.0,0.008...|
|751.0|    61.0|0.009600000000000004|22.0|           0.0|     0.0|       0.0|   3.0|    2|[751.0,61.0,0.009...|
|751.0|    61.0|0.008400000000000005|23.0|           0.0|     0.0|       0.0|   4.0|    2|[751.0,61.0,0.008...|
|751.0|    61.0|0.009600000000000004|24.0|           0.0|     0.0|       0.0|   5.0|    2|[751.0,61.0,0.

In [33]:
assembler = VectorAssembler(
    inputCols=('FICO', 'CLEANLTV','INCENTIVE','AGE','FIRSTTIMEBUYER','INVESTOR','SECONDHOME'),
    outputCol='features2')

df_f4 = assembler.transform(train)
df_f5 = assembler.transform(train)


In [23]:
assembler = VectorAssembler(
    inputCols=('FICO', 'CLEANLTV','INCENTIVE','AGE1','AGE2','FIRSTTIMEBUYER','INVESTOR','SECONDHOME','SEASON'),
    outputCol='features2')

df_f2 = assembler.transform(trainP)


assembler1 = VectorAssembler(
    inputCols=('FICO', 'CLEANLTV','INCENTIVE','AGE','AGE','FIRSTTIMEBUYER','INVESTOR','SECONDHOME','SEASON'),
    outputCol='features2')

df_f3 = assembler1.transform(df)





+-----+--------+--------------------+----+--------------+--------+----------+------+-----+--------------------+----+----+--------------------+
| FICO|CLEANLTV|           INCENTIVE| AGE|FIRSTTIMEBUYER|INVESTOR|SECONDHOME|SEASON|label|            features|AGE1|AGE2|           features2|
+-----+--------+--------------------+----+--------------+--------+----------+------+-----+--------------------+----+----+--------------------+
|751.0|    61.0|0.006200000000000004|19.0|           0.0|     0.0|       0.0|  12.0|    1|[751.0,61.0,0.006...|20.0|19.0|[751.0,61.0,0.006...|
|751.0|    61.0|0.008200000000000006|20.0|           0.0|     0.0|       0.0|   1.0|    1|[751.0,61.0,0.008...|20.0|20.0|[751.0,61.0,0.008...|
|751.0|    61.0|              0.0085|21.0|           0.0|     0.0|       0.0|   2.0|    2|[751.0,61.0,0.008...|21.0|20.0|[751.0,61.0,0.008...|
|751.0|    61.0|0.009600000000000004|22.0|           0.0|     0.0|       0.0|   3.0|    2|[751.0,61.0,0.009...|22.0|20.0|[751.0,61.0,0.009...|

In [5]:
def analyze_model(model, dataset):
	pred1 = model.transform(dataset)
	#pred1.groupby('prediction').count().show()
	#pred1.rdd.map(lambda x : -math.log(x["probability"][int(x["prediction"])])).map(lambda x : Row(x)).toDF().groupBy().avg().show()
	h1 = pred1.rdd.map(lambda x : -math.log(x["probability"][int(x["label"])])).map(lambda x : Row(x)).toDF().groupBy().avg().head()[0]
	h2 = pred1.rdd.map(lambda x : -sum(map(mul, map(math.log, map(lambda z: z + 1.0e-100, x["probability"])), x["probability"]))).map(lambda x : Row(float(x))).toDF().groupBy().avg().head()[0]
	return (h1, h2)

In [6]:
def check_one(name, fitter, ds1, ds2): 
	model = fitter.fit(ds1)
	(h1, h2) = analyze_model(model, ds1)
	(h3, h4) = analyze_model(model, ds2)
	print(name + ", " + str(h1) + ", " + str(h3) + ", " + str(h2) + ", " + str(h4))

In [34]:
def do_calc(ds1, ds2):

    #Entropy
    print("========= Entropy =========")
    fitter = LogisticRegression(labelCol="label", featuresCol="features2",maxIter=100, regParam=0.0001, elasticNetParam=0.0001)
    rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=5,  maxDepth=3,seed = 1)
    rf1 = RandomForestClassifier(labelCol="label", featuresCol="features2", numTrees=5,  maxDepth=3,seed=1)
    check_one("Entropy", fitter, ds1, ds2)
    check_one("Entropy", rf, ds1, ds2)
    check_one("Entropy", rf1, ds1, ds2)

In [35]:
df_in = df_f2.sample(False,100000/(1.0*df.count()),seed =1)
df_out = df_f3.sample(False,100000/(1.0*df.count()),seed =2)
do_calc(df_in, df_out)
#age 2 variables
#randon forest
#features and features2



========= Entropy =========
Entropy, 0.18914404008366115, 0.22701063313803096, 0.1866856017456471, 0.3051048331195982
Entropy, 0.1897972146213277, 0.19287326248368547, 0.193334537487614, 0.21460650006233023
Entropy, 0.1894288747986628, 0.19249134338696336, 0.19314775361291484, 0.20909734935873797


In [36]:
df_in = df_f4.sample(False,100000/(1.0*df.count()),seed =1)
df_out = df_f5.sample(False,100000/(1.0*df.count()),seed =2)
do_calc(df_in, df_out)
#season deletion
#features and features2
#


========= Entropy =========
Entropy, 0.1929848886307612, 0.1922622485380742, 0.1936120371641474, 0.19349052995964636
Entropy, 0.1897972146213277, 0.1892316679011291, 0.193334537487614, 0.1931591573319771
Entropy, 0.18977974294204664, 0.1893843070715667, 0.19269467211442673, 0.19248612722407496


In [ ]:
#season deletion
